In [1]:
!pip install torch trl sentencepiece safetensors
!pip install -U bitsandbytes
!pip install -U transformers datasets peft accelerate
!pip install datasets
!pip install peft
!pip install --upgrade peft
!pip install -U datasets
!pip install --upgrade transformers
!pip install wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import wandb
wandb.login(key="")



2025-06-13 10:39:42.143274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749811182.341702      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749811182.401152      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.

True

In [3]:

import torch
import torch.nn as nn
from transformers import (
    RobertaTokenizer, RobertaConfig, PreTrainedModel,
    Trainer, TrainingArguments, RobertaModel
)
from transformers.modeling_outputs import SequenceClassifierOutput
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
import numpy as np
import json
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)

ds = load_dataset("jingjietan/pandora-big5", split="train[:50000]")
ds = ds.train_test_split(test_size=0.2, seed=42)
train_ds = ds["train"]
val_ds = ds["test"]

class RobertaBig5Classifier(PreTrainedModel):
    config_class = RobertaConfig
    def __init__(self, config):
        super().__init__(config)
        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(config.hidden_size, 5)
    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        kwargs.pop("num_items_in_batch", None)
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(logits, labels)
        return SequenceClassifierOutput(loss=loss, logits=logits)

def preprocess(example):
    inputs = tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)
    labels = torch.tensor([example["O"], example["C"], example["E"], example["A"], example["N"]], dtype=torch.float) / 100.0
    inputs["labels"] = labels
    return inputs

train_ds = train_ds.map(preprocess, remove_columns=train_ds.column_names)
val_ds = val_ds.map(preprocess, remove_columns=val_ds.column_names)
train_ds.set_format(type="torch")
val_ds.set_format(type="torch")

learning_rates = [1e-5, 3e-5, 5e-5, 1e-4]
batch_sizes = [4, 8]

results = {}

for lr in learning_rates:
    for bs in batch_sizes:
        print(f"Training with learning_rate={lr}, batch_size={bs}")
        config = RobertaConfig.from_pretrained(model_name)
        model = RobertaBig5Classifier(config).to(device)
        model = prepare_model_for_kbit_training(model)
        lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=["query", "value"],
                                 lora_dropout=0.1, bias="none", task_type="SEQ_CLS")
        model = get_peft_model(model, lora_config)
        training_args = TrainingArguments(
            output_dir=f"./lora-big5-lr{lr}-bs{bs}",
            per_device_train_batch_size=bs,
            gradient_accumulation_steps=2,
            num_train_epochs=3,
            learning_rate=lr,
            fp16=False,
            logging_steps=100,
            save_strategy="no",
            eval_strategy="no"
        )
        trainer = Trainer(model=model, args=training_args, train_dataset=train_ds, tokenizer=tokenizer)
        trainer.train()
        model.eval()
        model.to(device)

        y_true, y_pred = [], []
        for batch in val_ds:
            inputs = {k: batch[k].unsqueeze(0).to(device) for k in ["input_ids", "attention_mask"]}
            labels = batch["labels"].unsqueeze(0).to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                preds = torch.sigmoid(outputs.logits).cpu().numpy() * 100
                true = labels.cpu().numpy() * 100
                y_pred.append(preds[0])
                y_true.append(true[0])
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        trait_names = ["O", "C", "E", "A", "N"]
        rmse_per_trait = np.sqrt(np.mean((y_true - y_pred) ** 2, axis=0)).tolist()
        mae_per_trait = np.mean(np.abs(y_true - y_pred), axis=0).tolist()
        within_15 = np.mean(np.abs(y_true - y_pred) <= 15) * 100

        results[f"lr={lr}_bs={bs}"] = {
            "rmse": dict(zip(trait_names, rmse_per_trait)),
            "mae": dict(zip(trait_names, mae_per_trait)),
            "accuracy_within_15": float(within_15)
        }

with open("big5_hyperparameter_results.json", "w") as f:
    json.dump(results, f, indent=2)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

/tmp/ipykernel_18/1739093657.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args, train_dataset=train_ds, tokenizer=tokenizer)
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,0.441300
200,0.405600
300,0.345000
400,0.325500
500,0.291900
600,0.280900
700,0.274200
800,0.276600
900,0.254100
1000,0.253600


Training with learning_rate=0.0001, batch_size=8


/tmp/ipykernel_18/1739093657.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args, train_dataset=train_ds, tokenizer=tokenizer)
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,0.410300
200,0.371600
300,0.311400
400,0.289600
500,0.271500
600,0.261700
700,0.263500
800,0.247900
900,0.248300
1000,0.237600


In [4]:
print(1)

1


In [5]:

from google.colab import files
files.download('userwise_bias_frequencies1.csv')

FileNotFoundError: Cannot find file: userwise_bias_frequencies1.csv